# M5 Machine Learning Pt. 1

This notebook will explore the ability of different machine learning (multiple linear regression, polynomial regression, and random forest) to predict rain rate from the provided polarised dataset. 

Here are some required modules for this notebook: 
- `seaborn`
- `sklearn`

### 1. Data preparation (load, process, and split the data)

In [1]:
# import modules

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [2]:
# load the dataset

df = pd.read_csv('homework/radar_parameters.csv', parse_dates=True)

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18969 entries, 0 to 18968
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      18969 non-null  int64  
 1   Zh (dBZ)        18969 non-null  float64
 2   Zdr (dB)        18969 non-null  float64
 3   Ldr (dB)        18969 non-null  float64
 4   Kdp (deg km-1)  18969 non-null  float64
 5   Ah (dBZ/km)     18969 non-null  float64
 6   Adr (dB/km)     18969 non-null  float64
 7   R (mm/hr)       18969 non-null  float64
dtypes: float64(7), int64(1)
memory usage: 1.2 MB


In [3]:
# drop unwanted column

df.drop('Unnamed: 0', axis=1, inplace=True)


In [4]:
# check the data properties 

df.describe()

,Zh (dBZ),Zdr (dB),Ldr (dB),Kdp (deg km-1),Ah (dBZ/km),Adr (dB/km),R (mm/hr)
count,18969.000000,18969.000000,18969.000000,18969.000000,18969.000000,18969.000000,18969.000000
mean,31.294021,0.762979,-37.969272,0.080879,0.001829,0.000234,7.855561
std,6.496330,0.363489,3.277391,0.221018,0.003469,0.000822,8.569413
min,14.036426,0.285207,-44.849249,0.000697,0.000044,0.000002,0.309399
25%,26.720145,0.489184,-40.573505,0.011537,0.000482,0.000027,3.072614
50%,31.020280,0.677804,-38.113140,0.028640,0.000977,0.000069,5.622457
75%,35.597165,0.947020,-35.601404,0.073099,0.001970,0.000182,9.622175
max,57.400639,3.843941,-25.373718,5.060710,0.082511,0.027538,195.557062


In [5]:
# calculate Z value from the given dBZ data
# and adding to the data

df['Z'] = 10**(df['Zh (dBZ)']/10)


In [6]:
# check the new data

df

,Zh (dBZ),Zdr (dB),Ldr (dB),Kdp (deg km-1),Ah (dBZ/km),Adr (dB/km),R (mm/hr),Z
0,23.144878,0.418637,-41.757733,0.005395,0.000290,0.000012,2.393520,206.294563
1,22.737156,0.322850,-43.772069,0.005194,0.000360,0.000012,3.502699,187.808651
2,26.869826,0.330948,-43.577399,0.013385,0.000903,0.000030,8.627561,486.387732
3,28.540561,0.399480,-42.139731,0.018872,0.001036,0.000043,8.424447,714.588688
4,30.500127,0.543758,-39.763087,0.027438,0.001157,0.000064,8.189291,1122.051192
...,...,...,...,...,...,...,...,...
18964,31.515997,0.579955,-39.244229,0.034048,0.001417,0.000080,10.648020,1417.750266
18965,29.993334,0.567935,-39.399188,0.024134,0.001032,0.000057,7.981875,998.466291
18966,31.685913,0.655681,-38.375696,0.033971,0.001165,0.000081,6.822691,1474.318332
18967,32.980096,0.768586,-37.166218,0.043117,0.001285,0.000105,6.801169,1986.138902


In [7]:
# new calculated baseline R number 

df['R_baseline'] = (df['Z']/200) ** (1/1.6)

In [8]:
# check the new dataset

df

,Zh (dBZ),Zdr (dB),Ldr (dB),Kdp (deg km-1),Ah (dBZ/km),Adr (dB/km),R (mm/hr),Z,R_baseline
0,23.144878,0.418637,-41.757733,0.005395,0.000290,0.000012,2.393520,206.294563,1.019556
1,22.737156,0.322850,-43.772069,0.005194,0.000360,0.000012,3.502699,187.808651,0.961454
2,26.869826,0.330948,-43.577399,0.013385,0.000903,0.000030,8.627561,486.387732,1.742691
3,28.540561,0.399480,-42.139731,0.018872,0.001036,0.000043,8.424447,714.588688,2.216365
4,30.500127,0.543758,-39.763087,0.027438,0.001157,0.000064,8.189291,1122.051192,2.938422
...,...,...,...,...,...,...,...,...,...
18964,31.515997,0.579955,-39.244229,0.034048,0.001417,0.000080,10.648020,1417.750266,3.400996
18965,29.993334,0.567935,-39.399188,0.024134,0.001032,0.000057,7.981875,998.466291,2.731742
18966,31.685913,0.655681,-38.375696,0.033971,0.001165,0.000081,6.822691,1474.318332,3.485185
18967,32.980096,0.768586,-37.166218,0.043117,0.001285,0.000105,6.801169,1986.138902,4.198675


In [9]:
# calculate r2 score of the available rain rate and rain rate from baseline equation

r2_baseline = r2_score(df['R (mm/hr)'].ravel(), df['R_baseline'])
r2_baseline

0.3023229070437503

In [10]:
# calculate rmse for baseline approach

rmse_baseline  = np.sqrt(mean_squared_error(df['R (mm/hr)'].ravel(), df['R_baseline']))
rmse_baseline

7.157590840042378

In [11]:
# drop any rows with null values

df.dropna(axis=0, how='any', inplace=True)

column_names = ['Zh (dBZ)', 'Zdr (dB)', 'Ldr (dB)', 'Kdp (deg km-1)', 'Ah (dBZ/km)', 'Adr (dB/km)']

X = df[column_names]  # hyperparameters
y = df['R (mm/hr)'] # target

In [12]:
# split the dataset

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0,
                                  train_size=0.7)                                               

### 2. Multiple Linear Regression

In [13]:
# train the model use training dataset created from number 1

from sklearn.linear_model import LinearRegression

mod_regres = LinearRegression(fit_intercept=True)
mod_regres.fit(X_train, y_train)

LinearRegression()

In [49]:
# apply model for test data set 

R_lin_regres = mod_regres.predict(X_test)

In [56]:
# calculate r2 score for multiple linear regression

r2_lin_regres_train = r2_score(y_test.ravel(), R_lin_regres)
r2_lin_regres_train

0.9868599917483046

In [57]:
# calculate root mean square error for multiple linear regression

rmse_in_regres_train  = np.sqrt(mean_squared_error(y_test.ravel(), R_lin_regres))
rmse_in_regres_train

0.9583564653829832

### 3. Grid Search over Polynomials orders and cross-validation

In [13]:
# import module and define the model

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

def PolynomialRegression(degree=np.arange(7), **kwargs):
    return make_pipeline(PolynomialFeatures(degree),
                         LinearRegression(**kwargs))

In [14]:
# set the hyperparameter and cross validation for polinom model

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures

param_grid = {'polynomialfeatures__degree': np.arange(7),
              'linearregression__fit_intercept': [True, False],
              'linearregression__normalize': [True, False]}

grid = GridSearchCV(PolynomialRegression(), param_grid, cv=7)


In [ ]:
# train the model with train dataset

grid.fit(X_train, y_train)

![grid](figures/grid-search.png)

In [16]:
grid.best_params_

{'linearregression__fit_intercept': True,
 'linearregression__normalize': False,
 'polynomialfeatures__degree': 2}

In [ ]:
mod_polinom = grid.best_estimator_

R_grid_poli = mod_polinom.fit(X_train, y_train).predict(X_test)

In [18]:
# calculate r2 score for multiple linear regression

r2_grid_poli = r2_score(y_test.ravel(), R_grid_poli)
r2_grid_poli

0.9994339502731618

In [19]:
# calculate root mean square error for random forest model

rmse_grid_poli  = np.sqrt(mean_squared_error(y_test.ravel(), R_grid_poli))
rmse_grid_poli

0.198909969449332

### 4. Random Forest Regressor

In [40]:
# import module

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

# create list of each hyper-parameter

n_estimators_list = [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
max_features_list = ['auto', 'sqrt']
max_depth_list = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None]
min_samples_split_list = [2, 5, 10]
min_samples_leaf_list = [1, 2, 4]
bootstrap_list = [True, False]


In [41]:
# structure model hyper-params as a dictionary

rf_grid = {'n_estimators': n_estimators_list,
           'max_features': max_features_list,
           'max_depth': max_depth_list,
           'min_samples_split': min_samples_split_list,
           'min_samples_leaf': min_samples_leaf_list,
           'bootstrap': bootstrap_list}

In [42]:
# reate base LGBM model

rf_base = RandomForestRegressor(random_state=42)

In [46]:
# create random search for LGBM model

rf_random = RandomizedSearchCV(estimator=rf_base, param_distributions=rf_grid, 
                                 n_iter=2, cv=7, verbose=2, random_state=42, 
                                 n_jobs=-1)

In [ ]:
%%time
# fit the random search LGBM model

rf_random.fit(X_train, y_train)

In [ ]:
# save the fitted random forest model 

import joblib

filename = 'finalized_model.sav'
joblib.dump(rf_random, filename)

In [ ]:
# restore the saved random forest model 

rf_random = joblib.load(filename)

In [ ]:
# get optimal hyper-params

rf_random.best_params_

In [ ]:
# get score of best model during hyper-param tuning

rf_random.best_score_

In [ ]:
# train model using optimal hyper-parameters from above steps

mod_rf = RandomForestRegressor(**rf_random.best_params_, random_state=42)

In [ ]:
# fit the best rf model to train dataset

mod_rf.fit(X_train, y_train)

In [ ]:
# predict R using rf model 

R_rf = mod_rf.predict(X_test)

In [ ]:
# calculate r2 score for random forest model

r2_rf = r2_score(y_test.ravel(), R_rf)
r2_rf

In [ ]:
# calculate root mean square error for random forest model

rmse_rf  = np.sqrt(mean_squared_error(y_test.ravel(), R_rf))
rmse_rf